# Лабораторная работа №3
## по предмету "Системы искусственного интеллекта"
### Выполнила студентка группы БВТ2101 Пьянова Анна

Целью работы является изучение методов регуляризации.


### Задание 1

Выгрузите данные из датасета. Изучите колонки, проверьте наличие пропусков. Постройте матрицу корреляции между признаками и целевой переменной. Сделайте выводы, что показывает эта матрица.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('processor.csv')
data.head()

,brand,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,os_bit,graphic_card_gb,weight,warranty,Touchscreen,msoffice,Price,rating,Number of Ratings,Number of Reviews
0,ASUS,Intel,Core i3,10th,4 GB,DDR4,0 GB,1024 GB,Windows,64-bit,0 GB,Casual,No warranty,No,No,34649,2 stars,3,0
1,Lenovo,Intel,Core i3,10th,4 GB,DDR4,0 GB,1024 GB,Windows,64-bit,0 GB,Casual,No warranty,No,No,38999,3 stars,65,5
2,Lenovo,Intel,Core i3,10th,4 GB,DDR4,0 GB,1024 GB,Windows,64-bit,0 GB,Casual,No warranty,No,No,39999,3 stars,8,1
3,ASUS,Intel,Core i5,10th,8 GB,DDR4,512 GB,0 GB,Windows,32-bit,2 GB,Casual,No warranty,No,No,69990,3 stars,0,0
4,ASUS,Intel,Celeron Dual,Not Available,4 GB,DDR4,0 GB,512 GB,Windows,64-bit,0 GB,Casual,No warranty,No,No,26990,3 stars,0,0


In [3]:
data.isnull().sum()

brand                0
processor_brand      0
processor_name       0
processor_gnrtn      0
ram_gb               0
ram_type             0
ssd                  0
hdd                  0
os                   0
os_bit               0
graphic_card_gb      0
weight               0
warranty             0
Touchscreen          0
msoffice             0
Price                0
rating               0
Number of Ratings    0
Number of Reviews    0
dtype: int64

In [4]:
data.select_dtypes(include=['int64', 'float64']).corr()

,Price,Number of Ratings,Number of Reviews
Price,1.000000,-0.140392,-0.148738
Number of Ratings,-0.140392,1.000000,0.989626
Number of Reviews,-0.148738,0.989626,1.000000


Предобрабработка данных

In [22]:
X = data.drop('Price', axis=1)
y = data['Price']

# Определение числовых и категориальных признаков
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Препроцессор для числовых признаков
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# Препроцессор для категориальных признаков
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

# Объединение препроцессоров
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Применение препроцессора
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

### Задание 2

Реализуйте с нуля алгоритм линеной регрессии  без использования регуляризации.

In [31]:
class LinearRegression:
    def __init__(self):
        self.weights = None
    
    def fit(self, X, y):
        # Добавляем столбец единиц для учета свободного члена (bias)
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        
        # Вычисляем веса по формуле МНК
        self.weights = np.linalg.inv(X.T @ X) @ X.T @ y
    
    def predict(self, X):
        # Добавляем столбец единиц для учета свободного члена (bias)
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        
        # Предсказываем значения
        return X @ self.weights
    
    def score(self, X, y):
        # Вычисляем RMSE
        y_pred = self.predict(X)
        return mean_squared_error(y, y_pred)

Обучите модель.

In [32]:
model = LinearRegression()
model.fit(X_train, y_train)

# Оценка модели
train_score = model.score(X_train, y_train)
val_score = model.score(X_val, y_val)
test_score = model.score(X_test, y_test)

print(f"Train MSE: {train_score:.2f}")
print(f"Validation MSE: {val_score:.2f}")
print(f"Test MSE: {test_score:.2f}")

Train MSE: 858188232903408.12
Validation MSE: 5128161810703051267191134887013836128256.00
Test MSE: 1442148356233143.50


### Задание 3

Реализуйте с нуля алгоритм линейной регрессии для многоклассовой классификации с L1-регуляризацией.

In [25]:
class L1LinearRegression:
    def __init__(self, alpha=1.0, learning_rate=0.01, n_iterations=1000):
            self.alpha = alpha  # Коэффициент регуляризации
            self.learning_rate = learning_rate  # Скорость обучения
            self.n_iterations = n_iterations  # Количество итераций
            self.weights = None
    
    def fit(self, X, y):
        # Добавляем столбец единиц для учета свободного члена (bias)
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        
        # Инициализируем веса случайными значениями
        self.weights = np.random.randn(X.shape[1])
        
        # Градиентный спуск
        for _ in range(self.n_iterations):
            gradient = self._compute_gradient(X, y)
            self.weights -= self.learning_rate * gradient

    def _compute_gradient(self, X, y):
        y_pred = X @ self.weights
        error = y_pred - y
        gradient = (X.T @ error) / len(y) + self.alpha * np.sign(self.weights)
        return gradient

    def predict(self, X):
        # Добавляем столбец единиц для учета свободного члена (bias)
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        
        # Предсказываем значения
        return X @ self.weights

    def score(self, X, y):
        # Вычисляем RMSE
        y_pred = self.predict(X)
        return mean_squared_error(y, y_pred)

Обучите модель.

In [26]:
model = L1LinearRegression()
model.fit(X_train, y_train)

# Оценка модели
train_score = model.score(X_train, y_train)
val_score = model.score(X_val, y_val)
test_score = model.score(X_test, y_test)

print(f"Train MSE: {train_score:.2f}")
print(f"Validation MSE: {val_score:.2f}")
print(f"Test MSE: {test_score:.2f}")

Train MSE: 708454746.98
Validation MSE: 556121526.72
Test MSE: 773901757.22


### Задание 4

Реализуйте с нуля алгоритм линейной регрессии для многоклассовой классификации с L2-регуляризацией.

In [27]:
class L2LinearRegression:
  def __init__(self, alpha=1.0, learning_rate=0.01, n_iterations=1000):
        self.alpha = alpha
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
    
  def fit(self, X, y):
      # Добавляем столбец единиц для учета свободного члена (bias)
      X = np.hstack([np.ones((X.shape[0], 1)), X])
      
      # Инициализируем веса случайными значениями
      self.weights = np.random.randn(X.shape[1])
      
      # Градиентный спуск
      for _ in range(self.n_iterations):
          gradient = self._compute_gradient(X, y)
          self.weights -= self.learning_rate * gradient
  
  def _compute_gradient(self, X, y):
      y_pred = X @ self.weights
      error = y_pred - y
      gradient = (X.T @ error) / len(y) + 2 * self.alpha * self.weights
      return gradient
  
  def predict(self, X):
      # Добавляем столбец единиц для учета свободного члена (bias)
      X = np.hstack([np.ones((X.shape[0], 1)), X])
      
      # Предсказываем значения
      return X @ self.weights
  
  def score(self, X, y):
      # Вычисляем RMSE
      y_pred = self.predict(X)
      return mean_squared_error(y, y_pred)

Обучите модель.

In [28]:
model = L2LinearRegression()
model.fit(X_train, y_train)

# Оценка модели
train_score = model.score(X_train, y_train)
val_score = model.score(X_val, y_val)
test_score = model.score(X_test, y_test)

print(f"Train MSE: {train_score:.2f}")
print(f"Validation MSE: {val_score:.2f}")
print(f"Test MSE: {test_score:.2f}")

Train MSE: 2040245298.25
Validation MSE: 1941948540.21
Test MSE: 1984496320.68


### Задание 5

Из результатов обучения можно сделать вывод, что лучшей оказалась модель линейной регрессии с L1-регуляризацией, так как средне-квадратичная ошибка на тренировочной, валидационной и тестовой выборках оказалась минимальной.